<a href="https://colab.research.google.com/github/m0hit-kumar/RASA/blob/main/BERT-Classfication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip Corrected_data.zip


Archive:  Corrected_data.zip
   creating: Corrected_data/
  inflating: Corrected_data/BombBlast(categorised).json  
  inflating: Corrected_data/Earthquake(categorised).json  
  inflating: Corrected_data/flood(categorised).json  
  inflating: Corrected_data/tornado(categorised).json  
  inflating: Corrected_data/tsnuami (categorised).json  
  inflating: Corrected_data/Volcano(categorised).json  
  inflating: Corrected_data/war(categorised).json  


In [47]:
import json
import os

all_data = []
for filename in os.listdir('Corrected_data/'):
    if filename.endswith(".json"):
        file_path = os.path.join('Corrected_data/', filename)
        with open(file_path, 'r') as file:
            data = json.load(file)
            all_data.extend(data)

with open("Data.json", 'w') as outfile:
    json.dump(all_data, outfile, indent=4)

In [48]:
import pandas as pd


In [49]:
data=pd.read_json("Data.json")

In [50]:
data["intent"].unique()


array(['Tornado', 'Miscellaneous', 'Miscellanous', 'Miscellenous',
       'Miscelleneous', 'BombBlast', 'Micscellaneous', 'Micsellaneous',
       'War', 'Micellaneous', 'miscellaneous', 'Miscellaeous',
       'Earthquake', nan, 'floods', 'Miscellaenous', 'Miscellneous',
       'Volcano'], dtype=object)

In [51]:
# checlking for nan value
data['intent'].fillna('misc', inplace=True)

data.loc[data["intent"].str.startswith("m"),"intent"]="misc"
data.loc[data["intent"].str.startswith("M"),"intent"]="misc"

In [52]:
data["intent"].unique()


array(['Tornado', 'misc', 'BombBlast', 'War', 'Earthquake', 'floods',
       'Volcano'], dtype=object)

In [53]:
data=data.to_dict(orient='records')
type(data)

list

In [54]:
data

[{'text': 'Four miles north of Herington, Kansas.” A close friend of the victims and fellow meteorology student at University of Oklahoma said that they had all been so excited to cover the storms with Mr Brooks even seeing a tornado for the first time.At this time, we ask the public to respect the privacy of their families.” The College of Atmospheric and Geographic Sciences at the University of Oklahoma also paid tribute to the students saying its members were “deeply saddened” by their tragic deaths.',
  'intent': 'Tornado'},
 {'text': 'WICHITA, Kansas -- A tornado that barreled through parts of Kansas destroyed or damaged hundreds of homes and buildings, injured several people and left more than 15,000 people without power, officials said Saturday.In addition to wreckage from the tornado itself, three University of Oklahoma meteorology students traveling back from storm chasing in Kansas were also killed in a crash Friday evening, according to officials.Nicholas Nair, 20, of Denton

In [55]:
import nltk
nltk.download('stopwords') # for removing useless data
nltk.download('punkt') # use unsupervised algorithm to build a model for abbreviation words, collocations, and words
nltk.download('wordnet') #used  in information systems, word-sense disambiguation, information retrieval, automatic text classification, summarization, machine translation.
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [56]:
# clean the function will apply the above mentioned preprocessing to the resumes
def clean(text):
    wn = nltk.WordNetLemmatizer()
    stopword = nltk.corpus.stopwords.words('english')
    tokens = nltk.word_tokenize(text)
    lower = [word.lower() for word in tokens]
    no_stopwords = [word for word in lower if word not in stopword]
    no_alpha = [word for word in no_stopwords if word.isalpha()]
    lemm_text = [wn.lemmatize(word) for word in no_alpha]
    clean_text = lemm_text
    # make list of text into string
    result=" ".join(clean_text)
    return result


In [57]:
!pip install transformers
!pip install torch

In [62]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig, AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split

In [64]:
for i in data:
  i["text"]=clean(i["text"])

In [ ]:
 # Example data
# data = [
#     {"text": "Floods are hitting the coastal areas", "intent": "flood"},
#     {"text": "Tornado warning issued for the city", "intent": "tornado"},
#     {"text": "Earthquake devastates the region", "intent": "earthquake"},
#     # more data here
# ]

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=7)


In [66]:
intent_label_map={'Tornado':0, 'misc':1, 'BombBlast':2, 'War':3, 'Earthquake':4, 'floods':5,'Volcano':6}

In [ ]:

# Tokenize the data
def tokenize_data(data):
    inputs = tokenizer(data["text"], padding=True, truncation=True, return_tensors="pt")
    labels = torch.tensor([intent_label_map[data["intent"]]])  # Convert intent string to numerical label
    return inputs, labels

train_data_tokenized = [tokenize_data(d) for d in train_data]
test_data_tokenized = [tokenize_data(d) for d in test_data]

# Define the optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=1e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_data))

# Training loop
epochs = 3
for epoch in range(epochs):
    for batch in train_data_tokenized:
        optimizer.zero_grad()
        outputs = model(**batch[0], labels=batch[1])
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

# Evaluation
model.eval()
total = 0
correct = 0
with torch.no_grad():
    for batch in test_data_tokenized:
        outputs = model(**batch[0])
        predicted = torch.argmax(outputs.logits, dim=1)
        total += batch[1].size(0)
        correct += (predicted == batch[1]).sum().item()

accuracy = correct / total
print(f"Accuracy: {accuracy}")

# Save the model
output_dir = './model_save/'
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"Model saved in {output_dir}")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
